In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class GAT(torch.nn.Module):
    def __init__(self, nfeat, nclass):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        
        self.conv1 = GATConv(nfeat, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, nclass, concat=False,
                             heads=self.out_head, dropout=0.6)
    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [5]:
from Lib import *

def run(name_data, epochs, print_it, config):
    # The data characteristics
    dataset= load_data(name_data)
    print(dataset[0])
    print(f"Number of Classes in {name_data}:", dataset.num_classes)
    print(f"Number of Node Features in {name_data}:", dataset.num_node_features)
    n_Tasks=dataset.num_classes
    x = dataset[0].x
    y = dataset[0].y
    edge_index = dataset[0].edge_index 
    continuum_data = continuum_node_classification(dataset, n_Tasks, num_classes=dataset.num_classes)

    # The model use GCN
    # model = GCN(hidden_channels=16, num_features=dataset.num_features, n_classes=dataset.num_classes)
    # The Gat model

    model = GAT(nfeat=dataset.num_node_features, 
                nclass=dataset.num_classes)


    criterion = torch.nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    # The arrays for data
    memory_train=[]
    memory_test=[]
    memory_valid=[]
    accuracies_mem = []
    accuracies_one=[]
    Total_loss=[]
    Gen_loss=[]
    For_loss=[]

    for id, task in enumerate(continuum_data):
        train_mask, _, test_mask = task
        memory_train.append(train_mask)
        memory_test.append(test_mask)
        for epoch in range(epochs):
            # loss = train(model, Data(x=x,edge_index=edge_index, y=y, train_mask=train_mask),\
            #       optimizer, criterion)
            # print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
            train_loader= Data(x=x,edge_index=edge_index, y=y, train_mask=train_mask)
            Total,Gen,For=train_CL( model, criterion, optimizer, memory_train, train_loader, task=id, \
                graph = 0, node=1, params = config)
            Total_loss.append(Total)
            Gen_loss.append(Gen)
            For_loss.append(For)

            if epoch%print_it==0:
                # scheduler.step()
                train_acc = test_NC(model, train_loader.x, train_loader.edge_index, train_mask, train_loader.y)
                test_acc = test_NC(model,  train_loader.x, train_loader.edge_index, test_mask, train_loader.y)
                mem_train_acc= [ test_NC(model, train_loader.x, train_loader.edge_index, element, train_loader.y) for element in memory_train]
                mem_train_acc = sum(mem_train_acc)/len(memory_train)
                mem_test_acc= [ test_NC(model, train_loader.x, train_loader.edge_index, element, train_loader.y) for element in memory_test]
                mem_test_acc  = sum(mem_test_acc)/len(memory_test)
                accuracies_mem.append(mem_test_acc)
                accuracies_one.append(test_acc)
                print("#########################################################################")
                print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
                print(f'Mem Train Acc: {mem_train_acc:.4f}, Mem Test Acc: {mem_test_acc:.4f}')
                print("#########################################################################")

    import matplotlib.pyplot as plt
    plt.plot(accuracies_mem, label='memory accuracy')
    plt.plot(accuracies_one, label='task accuracy')
    return accuracies_mem, accuracies_one

run('Cora', epochs=100, print_it=50, config={'x_updates': 10,  'theta_updates': 10, 'factor': 1, 'x_lr': 0.001,'th_lr':0.001,\
                'device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),\
                'batchsize':8, 'total_updates': 1000} )

from the load dataset tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of Classes in Cora: 7
Number of Node Features in Cora: 1433
features inside the start tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
lable, n_task 7 1
13 20
69 130
13 20
69 130
#########################################################################
Epoch: 000, Train Acc: 0.6500, Test Acc: 0.5308
Mem Train Acc: 0.650